In [1]:
import sys
from logHelper import logger

In [11]:
#Import the necessary modules
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ast import literal_eval

In [3]:
#Import the necessary modules for NLP
! pip install nltk
import nltk

Unable to create process using 'C:\Users\Kartheek Kotha\.conda\envs\nlp\python.exe "C:\Users\Kartheek Kotha\.conda\envs\nlp\Scripts\pip-script.py" install nltk'


## Configurations

In [8]:
base_config = {
    "Base_data_path": "../data/nbme-score-clinical-patient-notes"
}

## Helper Classes

### To prepare data

In [35]:
class prepare_data():
    def __init__(self , config):
        self.config = config
    def merge_data(self):
        features = pd.read_csv(f"{self.config['Base_data_path']}/features.csv")
        patient_notes = pd.read_csv(f"{self.config['Base_data_path']}/patient_notes.csv")
        train_df = pd.read_csv(f"{self.config['Base_data_path']}/train.csv")
        train_df['annotation_list'] = [literal_eval(x) for x in train_df['annotation']]
        train_df['location_list'] = [literal_eval(x) for x in train_df['location']]
        
        merged = train_df.merge(patient_notes, how='left')
        merged = merged.merge(features, how='left')

        merged['pn_history'] = merged['pn_history'].apply(lambda x: x.lower())
        merged['feature_text'] = merged['feature_text'].apply(lambda x: x.lower())
        merged['feature_text'] = merged['feature_text'].apply(lambda x: x.replace('-', ' ').replace('-OR-', ";-"))
        return merged

### To pre-process data - Tokenizer

In [54]:
class pre_process_data():
    def __init__(self , config):
        self.config = config
    def split_loc(self , loc_list):
        final_loc = []
        for loc in loc_list:
            locations = loc.split(';')
            for location in locations:
                start , end = location.split()
                final_loc.append((int(start) , int(end)))
        return final_loc

## Testing purpose below

In [55]:
#read data
df = pd.read_csv(f"{base_config['Base_data_path']}/train.csv")
print(df.info())
df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14300 entries, 0 to 14299
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           14300 non-null  object
 1   case_num     14300 non-null  int64 
 2   pn_num       14300 non-null  int64 
 3   feature_num  14300 non-null  int64 
 4   annotation   14300 non-null  object
 5   location     14300 non-null  object
dtypes: int64(3), object(3)
memory usage: 670.4+ KB
None


,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']


In [56]:
obj = prepare_data(base_config)
train_df = obj.merge_data()
print(train_df.info())
train_df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14300 entries, 0 to 14299
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               14300 non-null  object
 1   case_num         14300 non-null  int64 
 2   pn_num           14300 non-null  int64 
 3   feature_num      14300 non-null  int64 
 4   annotation       14300 non-null  object
 5   location         14300 non-null  object
 6   annotation_list  14300 non-null  object
 7   location_list    14300 non-null  object
 8   pn_history       14300 non-null  object
 9   feature_text     14300 non-null  object
dtypes: int64(3), object(7)
memory usage: 1.1+ MB
None


,id,case_num,pn_num,feature_num,annotation,location,annotation_list,location_list,pn_history,feature_text
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],[dad with recent heart attcak],[696 724],hpi: 17yo m presents with palpitations. patien...,family history of mi or family history of myoc...
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693'],"[mom with ""thyroid disease]",[668 693],hpi: 17yo m presents with palpitations. patien...,family history of thyroid disorder


In [59]:
obj_pre_process = pre_process_data(base_config)
print(obj_pre_process.split_loc(train_df['location_list'][0]))

[(696, 724)]
